In [ ]:
#interval calculation with HRV feature extraction and binning

import hrvanalysis as hrv
import scipy.signal as signal
def cal_hrv(beat_interval):
    #remove outliers
    print("inital length")

    filtered_interval = hrv.remove_outliers(beat_interval,low_rri = 400,high_rri = 1500)
    #Replace na values with linear interpolation using forier transfer
    interpolated_interval = hrv.interpolate_nan_values(filtered_interval,interpolation_method="linear")
    # This remove ectopic beats from signal
    nn_intervals_list = hrv.remove_ectopic_beats(interpolated_interval, method="malik")
    # This replace ectopic beats nan values with linear interpolation
    interpolated_nn_intervals = hrv.interpolate_nan_values(nn_intervals_list)
    return interpolated_nn_intervals

def hrv_generation(peak_intervals,binsize,orign_df, mechano_acoustic = True):
    """ This function takes a peak peak interval list, bin size in second,a boolean, and the original dataset with time stamps as input, then will remove the abnormal intervals and heart beats and generate the HRV features.  """
   
    if mechano_acoustic:
        ma_df = orign_df
        peaks, _ = signal.find_peaks(peak_intervals,height=0.2*10**-5,distance=400)
        itv_1 = peaks[0]
        i = 0
        k = 0
        hr = pd.DataFrame(columns=['index','Hr'])
        ma_it = pd.DataFrame(columns=['index','Hr'])
        hrv_feature = pd.DataFrame()
        while i < peaks.size-2:

            interval = peaks[i+2]-itv_1
            # itv_1 = peaks[i+2]
            # i = i+2
            if interval > 1200:
                interval = peaks[i+1]-itv_1
                itv_1 = peaks[i+1]
                i = i+1
            else: 
                itv_1 = peaks[i+2]
                i = i+2

            k=k+interval
            hr_df = pd.DataFrame({'index': itv_1,'Hr':interval},index = [itv_1])
            hr = pd.concat([hr,hr_df])
            ma_it = pd.concat([ma_it,hr_df])
            if k > binsize*1000:
                bined_interval = hr.Hr.values.tolist()
                filtered_interval = cal_hrv(bined_interval)
                try:
                    feature = pd.DataFrame([hrv.get_frequency_domain_features(filtered_interval)])
                except:
                    feature = pd.DataFrame([hrv.get_frequency_domain_features(bined_interval)])
                feature['time'] = ma_df.index[itv_1]
                hrv_feature = pd.concat([hrv_feature,feature],ignore_index=True)
                hr = pd.DataFrame(columns=['index','Hr'])
                k=0 
            
        ma_hrv = hrv_feature.copy()
        return ma_hrv, ma_it

    
    else:
        # ECG data
        ecg_df = peak_intervals
        peaks, _ = signal.find_peaks(ecg_df,height=200,distance=350)

        itv_1 = peaks[0]
        i = 0
        time = 0
        k=0
        hr = pd.DataFrame(columns=['index','Hr'])
        ecg_it = pd.DataFrame(columns=['index','Hr'])
        hrv_feature = pd.DataFrame()
        while i < peaks.size-2:

            interval = peaks[i+1]-itv_1
            itv_1 = peaks[i+1]
            i = i+1


            hr_df = pd.DataFrame({'index': itv_1,'Hr':interval},index = [itv_1])
            hr = pd.concat([hr,hr_df])
            ecg_it = pd.concat([ecg_it,hr_df])
            time = time+interval
            k=k+interval

        

            if k > binsize*1000:
                bined_interval = hr.Hr.values.tolist()
                filtered_interval = cal_hrv(bined_interval)
                hr = pd.DataFrame(columns=['index','Hr'])
                try:
                    feature = pd.DataFrame([hrv.get_frequency_domain_features(filtered_interval)])
                except:
                    feature = pd.DataFrame([hrv.get_frequency_domain_features(bined_interval)])
                feature['time'] = ecg_df.index[time]
                hrv_feature = pd.concat([hrv_feature,feature],ignore_index=True)
                k=0 


        hr.index = hr['index']

        # # hr = 1000/hr*60
        # ecg_it = hr[(hr['Hr']<1200) & (hr['Hr']>750)]
        ecg_hrv = hrv_feature.copy()
        return ecg_hrv, ecg_it
    
            